In [2]:

import os
from google.cloud import bigquery
import pandas as pd
from datetime import datetime

# CONFIGURATION
PROJECT_ID = "patent-risk-250624"
client = bigquery.Client(project=PROJECT_ID)

print("=" * 80)
print("PATENT DATASET EXPLORATION")
print("=" * 80)

PATENT DATASET EXPLORATION


In [ ]:

# =============================================================================
# QUESTION 1: How does the date system work?
# =============================================================================
print("\n" + "=" * 80)
print("QUESTION 1: DATE SYSTEM ANALYSIS")
print("=" * 80)

# 1.1 Get sample dates to understand format
query_date_samples = """
SELECT 
    publication_date,
    MIN(publication_date) as min_date,
    MAX(publication_date) as max_date,
    COUNT(DISTINCT publication_date) as unique_dates
FROM 
    `patents-public-data.patents.publications`
GROUP BY 
    publication_date
ORDER BY 
    publication_date DESC
LIMIT 10000
"""

print("\n1.1 Sample of recent publication dates:")
df_dates = client.query(query_date_samples).to_dataframe()
print(df_dates)
print(f"\nDate format appears to be: YYYYMMDD (integer)")

# 1.2 Get overall date range
query_date_range = """
SELECT 
    MIN(publication_date) as earliest_date,
    MAX(publication_date) as latest_date,
    COUNT(DISTINCT publication_date) as total_unique_dates,
    COUNT(*) as total_records
FROM 
    `patents-public-data.patents.publications`
"""

print("\n1.2 Overall date range in dataset:")
df_range = client.query(query_date_range).to_dataframe()
print(df_range)



QUESTION 1: DATE SYSTEM ANALYSIS

1.1 Sample of recent publication dates:


/home/jack/patent_risk_website_2/.venv_patents/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


      publication_date  min_date  max_date  unique_dates
0             20251023  20251023  20251023             1
1             20251022  20251022  20251022             1
2             20251021  20251021  20251021             1
3             20251017  20251017  20251017             1
4             20251016  20251016  20251016             1
...                ...       ...       ...           ...
9995          19980605  19980605  19980605             1
9996          19980604  19980604  19980604             1
9997          19980603  19980603  19980603             1
9998          19980602  19980602  19980602             1
9999          19980601  19980601  19980601             1

[10000 rows x 4 columns]

Date format appears to be: YYYYMMDD (integer)

1.2 Overall date range in dataset:


/home/jack/patent_risk_website_2/.venv_patents/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   earliest_date  latest_date  total_unique_dates  total_records
0              0     20251023               56863      166866499

Earliest: 0--
Latest: 2025-10-23


In [6]:

# =============================================================================
# QUESTION 2: How many files per day? Why none in 2026?
# =============================================================================
print("\n" + "=" * 80)
print("QUESTION 2: RECORDS PER DAY & 2026 DATA")
print("=" * 80)

# 2.1 Check for 2026 data specifically
query_2026 = """
SELECT 
    publication_date,
    COUNT(*) as record_count
FROM 
    `patents-public-data.patents.publications`
WHERE 
    publication_date >= 20260101
    AND publication_date < 20270101
GROUP BY 
    publication_date
ORDER BY 
    publication_date
"""

print("\n2.1 Checking for 2026 data:")
df_2026 = client.query(query_2026).to_dataframe()
if len(df_2026) == 0:
    print("NO DATA FOUND FOR 2026")
    print("This is expected - patents are published after filing and examination.")
    print("The dataset likely contains historical data up to recent past.")
else:
    print(df_2026)

# 2.2 Get records per day for recent years
query_daily_counts = """
SELECT 
    publication_date,
    COUNT(*) as record_count,
    EXTRACT(YEAR FROM PARSE_DATE('%Y%m%d', CAST(publication_date AS STRING))) as year,
    EXTRACT(MONTH FROM PARSE_DATE('%Y%m%d', CAST(publication_date AS STRING))) as month
FROM 
    `patents-public-data.patents.publications`
WHERE 
    publication_date >= 20240101
    AND publication_date < 20260101
GROUP BY 
    publication_date
ORDER BY 
    publication_date DESC
LIMIT 50
"""

print("\n2.2 Records per day (2024-2025):")
df_daily = client.query(query_daily_counts).to_dataframe()
print(df_daily)
print(f"\nAverage records per day: {df_daily['record_count'].mean():.0f}")
print(f"Min records per day: {df_daily['record_count'].min()}")
print(f"Max records per day: {df_daily['record_count'].max()}")

# 2.3 Check publication frequency by year
query_yearly = """
SELECT 
    EXTRACT(YEAR FROM PARSE_DATE('%Y%m%d', CAST(publication_date AS STRING))) as year,
    COUNT(*) as total_records,
    COUNT(DISTINCT publication_date) as days_with_data
FROM 
    `patents-public-data.patents.publications`
WHERE 
    publication_date >= 20200101
GROUP BY 
    year
ORDER BY 
    year DESC
"""

print("\n2.3 Records by year (2020+):")
df_yearly = client.query(query_yearly).to_dataframe()
print(df_yearly)


QUESTION 2: RECORDS PER DAY & 2026 DATA

2.1 Checking for 2026 data:


/home/jack/patent_risk_website_2/.venv_patents/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


NO DATA FOUND FOR 2026
This is expected - patents are published after filing and examination.
The dataset likely contains historical data up to recent past.

2.2 Records per day (2024-2025):
    publication_date  record_count  year  month
0           20251023          7971  2025     10
1           20251022          8144  2025     10
2           20251021          7614  2025     10
3           20251017         59263  2025     10
4           20251016         14363  2025     10
5           20251015         10025  2025     10
6           20251014         43903  2025     10
7           20251013           502  2025     10
8           20251012            17  2025     10
9           20251011             3  2025     10
10          20251010         49576  2025     10
11          20251009         18227  2025     10
12          20251008         10242  2025     10
13          20251007         11360  2025     10
14          20251006          1711  2025     10
15          20251005             2  2025 

/home/jack/patent_risk_website_2/.venv_patents/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   year  total_records  days_with_data
0  2025        5709355             293
1  2024        7770711             366
2  2023        7572818             365
3  2022        8132228             365
4  2021        8478151             365
5  2020        7344922             366


In [7]:
# =============================================================================
# QUESTION 3: What do the fields mean? Explore field values
# =============================================================================
print("\n" + "=" * 80)
print("QUESTION 3: FIELD EXPLORATION")
print("=" * 80)

# 3.1 Get schema information
query_schema = """
SELECT 
    column_name,
    data_type,
    is_nullable
FROM 
    `patents-public-data.patents.INFORMATION_SCHEMA.COLUMNS`
WHERE 
    table_name = 'publications'
ORDER BY 
    ordinal_position
"""

print("\n3.1 Table schema (all fields):")
df_schema = client.query(query_schema).to_dataframe()
print(df_schema.to_string())

# 3.2 Sample actual data to see field contents
query_sample = """
SELECT 
    *
FROM 
    `patents-public-data.patents.publications`
WHERE 
    publication_date >= 20250101
LIMIT 5
"""

print("\n3.2 Sample records (first 5 from 2025):")
df_sample = client.query(query_sample).to_dataframe()
print("\nColumns in dataset:")
for col in df_sample.columns:
    print(f"  - {col}")

# Show first record in detail
print("\n3.3 First record detailed view:")
if len(df_sample) > 0:
    first_record = df_sample.iloc[0]
    for col in first_record.index:
        value = first_record[col]
        # Truncate long values
        if isinstance(value, str) and len(value) > 100:
            value = value[:100] + "..."
        print(f"{col}: {value}")

# 3.3 Explore specific important fields
query_field_analysis = """
SELECT 
    country_code,
    COUNT(*) as count
FROM 
    `patents-public-data.patents.publications`
WHERE 
    publication_date >= 20240101
GROUP BY 
    country_code
ORDER BY 
    count DESC
LIMIT 10
"""

print("\n3.4 Country code distribution (2024+):")
df_countries = client.query(query_field_analysis).to_dataframe()
print(df_countries)

# Application types
query_application_types = """
SELECT 
    application_kind,
    COUNT(*) as count
FROM 
    `patents-public-data.patents.publications`
WHERE 
    publication_date >= 20240101
    AND application_kind IS NOT NULL
GROUP BY 
    application_kind
ORDER BY 
    count DESC
LIMIT 10
"""

print("\n3.5 Application types (2024+):")
df_app_types = client.query(query_application_types).to_dataframe()
print(df_app_types)



QUESTION 3: FIELD EXPLORATION

3.1 Table schema (all fields):


/home/jack/patent_risk_website_2/.venv_patents/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


                     column_name                                                                                                                              data_type is_nullable
0             publication_number                                                                                                                                 STRING         YES
1             application_number                                                                                                                                 STRING         YES
2                   country_code                                                                                                                                 STRING         YES
3                      kind_code                                                                                                                                 STRING         YES
4               application_kind                                                                    

/home/jack/patent_risk_website_2/.venv_patents/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Columns in dataset:
  - publication_number
  - application_number
  - country_code
  - kind_code
  - application_kind
  - application_number_formatted
  - pct_number
  - family_id
  - spif_publication_number
  - spif_application_number
  - title_localized
  - abstract_localized
  - claims_localized
  - claims_localized_html
  - description_localized
  - description_localized_html
  - publication_date
  - filing_date
  - grant_date
  - priority_date
  - priority_claim
  - inventor
  - inventor_harmonized
  - assignee
  - assignee_harmonized
  - examiner
  - uspc
  - ipc
  - cpc
  - fi
  - fterm
  - locarno
  - citation
  - parent
  - child
  - entity_status
  - art_unit

3.3 First record detailed view:
publication_number: CN-120648158-A
application_number: CN-202410293650-A
country_code: CN
kind_code: A
application_kind: A
application_number_formatted: CN 202410293650
pct_number: 
family_id: 96998133
spif_publication_number: CN120648158A
spif_application_number: CN202410293650
title_lo

/home/jack/patent_risk_website_2/.venv_patents/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


  application_kind    count
0                A  9422133
1                U  3256880
2                W   512949
3                F   172881
4                T   112481
5                V     1751
6                C      963
7                D       27
8                K        1


In [ ]:
# =============================================================================
# QUESTION 4: What other tables are in the dataset?
# =============================================================================
print("\n" + "=" * 80)
print("QUESTION 4: OTHER TABLES IN DATASET")
print("=" * 80)

query_tables = """
SELECT 
    table_id as table_name,
    type as table_type,
    ROUND(size_bytes / 1024 / 1024 / 1024, 2) as size_gb
FROM 
    `patents-public-data.patents.__TABLES__`
ORDER BY 
    table_id
"""

print("\n4.1 All tables in patents-public-data.patents:")
df_tables = client.query(query_tables).to_dataframe()
print(df_tables.to_string())

# Get row counts for each table
print("\n4.2 Row counts for each table:")
for table_name in df_tables['table_name']:
    try:
        count_query = f"""
        SELECT COUNT(*) as row_count
        FROM `patents-public-data.patents.{table_name}`
        """
        result = client.query(count_query).to_dataframe()
        print(f"  {table_name}: {result['row_count'].iloc[0]:,} rows")
    except Exception as e:
        print(f"  {table_name}: Error - {str(e)[:50]}")


QUESTION 4: OTHER TABLES IN DATASET

4.1 All tables in patents-public-data.patents:


BadRequest: 400 Unrecognized name: table_name at [3:5]; reason: invalidQuery, location: query, message: Unrecognized name: table_name at [3:5]

Location: US
Job ID: eb5f3f01-dae9-4c91-bc3d-d6b2d8d3f1e7
